In [1]:
# imports

import sqlalchemy
import pandas as pd

from sklearn import datasets
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from sklearn.preprocessing import RobustScaler

import pickle
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor


In [2]:
#STEP 1 IMPORT DATASET

diamonds_train = pd.read_csv('./dataset/diamonds_train.csv')
diamonds_test = pd.read_csv('./dataset/diamonds_test.csv')

In [21]:
#STEP 2 WRANGLING
#STEP 2.1 REMOVE COLUMN CITY 
diamonds_train = diamonds_train.drop(['city'], axis=1)

In [22]:
#STEP 3 CONVERT CATEGORIC VALUES INTO NUMERIC
#STEP 3.1 COLOR BECOMES 'color-encoded'
from sklearn.preprocessing import OrdinalEncoder

#STEP 3.1.1 CREATE ENCODER INDICATING ORDER OF VARIABLE (low VALUE TO high Value)
encoder = OrdinalEncoder(categories=[['J', 'I', 'H', 'G', 'F', 'E', 'D']])

#STEP 3.1.2 TRANSFORM VALUES IN COLUMN 'color' INTO NUMERIC VALUES IN TRAINING DATASET
encoder.fit(diamonds_train[['color']])
diamonds_train['color-encoded'] = encoder.transform(diamonds_train[['color']])
diamonds_train.drop('color', axis = 1 , inplace = True)

#STEP 3.1.3 TRANSFORM VALUES IN COLUMN 'color' INTO NUMERIC VALUES IN TEST DATASET
encoder.fit(diamonds_test[["color"]])
diamonds_test["color-encoded"] = encoder.transform(diamonds_test[["color"]])
diamonds_test.drop("color", axis = 1 , inplace = True)

In [23]:
#STEP 3.2 CUT BECOMES 'cut-encoded'

#STEP 3.2.1 CREATE ENCODER INDICATING ORDER OF VARIABLE (low VALUE TO high Value)
encoder = OrdinalEncoder(categories=[['Premium', 'Very Good', 'Fair', 'Good', 'Ideal']])

#STEP 3.2.2 TRANSFORM VALUES IN COLUMN 'cut' INTO NUMERIC VALUES IN TRAINING DATASET
encoder.fit(diamonds_train[['cut']])
diamonds_train['cut-encoded'] = encoder.transform(diamonds_train[['cut']])
diamonds_train.drop('cut', axis = 1 , inplace = True)

#STEP 3.2.3 TRANSFORM VALUES IN COLUMN 'cut' INTO NUMERIC VALUES IN TEST DATASET
encoder.fit(diamonds_test[['cut']])
diamonds_test['cut-encoded'] = encoder.transform(diamonds_test[['cut']])
diamonds_test.drop('cut', axis = 1 , inplace = True)

In [24]:
#STEP 3.3 CLARITY BECOMES 'clarity-encoded'

#STEP 3.3.1 CREATE ENCODER INDICATING ORDER OF VARIABLE (low VALUE TO high Value)
encoder = OrdinalEncoder(categories=[['VS2', 'SI1', 'SI2', 'VS1', 'IF', 'VVS1', 'VVS2', 'I1']])

#STEP 3.3.2 TRANSFORM VALUES IN COLUMN 'clarity' INTO NUMERIC VALUES IN TRAINING DATASET
encoder.fit(diamonds_train[['clarity']])
diamonds_train['clarity-encoded'] = encoder.transform(diamonds_train[['clarity']])
diamonds_train.drop('clarity', axis = 1 , inplace = True)

#STEP 3.3.3 TRANSFORM VALUES IN COLUMN 'clarity' INTO NUMERIC VALUES IN TEST DATASET
encoder.fit(diamonds_test[['clarity']])
diamonds_test['clarity-encoded'] = encoder.transform(diamonds_test[['clarity']])
diamonds_test.drop('clarity', axis = 1 , inplace = True)

In [25]:
#STEP 4 DEFINE COLUMN TARGET AND FEATURES (IN OUR MODEL, ALL THE FEATURES ARE NUMERICAL)

target = "price"
features = ['depth', 'carat', 'table', 'x', 'y', 'z', 'color-encoded', 'cut-encoded', 'clarity-encoded']

In [26]:
#STEP 5 APLY SCALER ON TRAININING MODEL

scaler = RobustScaler()
x =scaler.fit_transform(diamonds_train[features])
y = diamonds_train[target]



In [27]:
#STEP 6 SPLIT TEST MODEL TO APPLY OUR TRAINING DATASET
X_train, X_validation, y_train, y_validation=train_test_split(x, y, test_size=0.2, random_state=42)


In [28]:
#STEP 7 SELECT A MODEL OF REGRESSION (IN THIS CASE, RANDOMFOREST IS APPLIED) AND TRAIN THE MODEL
model1 = RandomForestRegressor(random_state=42)
model1.fit(X_train,y_train)

y_pred_train = model1.predict(X_validation)

In [29]:
#STEP 8 OBTAIN RMS TO CONFIRM THE HIGH VALUE OF OUR MODEL
RMSE_ERROR_sklearn = mean_squared_error(y_validation, y_pred_train, squared = False)
print(RMSE_ERROR_sklearn)

551.1019282650497


In [30]:
#STEP 9 APPLY SCALER IN TEST DATASET

scaler = RobustScaler()
scaler_test =scaler.fit_transform(diamonds_test[features])

y_pred_test = model1.predict(scaler_test).clip(0,30000)


In [31]:
#STEP 10 CREATE THE DATASET TO FACE KAGGLE MODEL
submission_test =pd.DataFrame({'id': diamonds_test['id'],'price': y_pred_test})
submission_test.head()

,id,price
0,0,2881.62
1,1,5355.66
2,2,9589.35
3,3,4173.35
4,4,1680.62


In [32]:
#STEP 11 EXPORT DATASET INTO CSV
submission_test.to_csv('./submission_test/submission_test_v13.csv', index =False)